## 16.03 Count the number of ways to traverse a 2D array

In this problem you are to count the number of ways starting at the top-left corner of a 2D array and getting to the bottom-right corner.  All moves must either go right or down.  For example, we show three ways in a 5 x 5 2D array in Figure 16.5. (As we will see, there are a total of 70 possible ways for this example.)

Write a program that counts how many ways you can go from the top-left to the bottom-right in a 2D array.

### Hint:

If i > 0, and j > 0, you can get to (i,j) from (i-i, j) or (i, j-1)

### Initial Remarks

There are two ways to get to the bottom right.

In [1]:
def how_many_traversals(i, j, start=False):
    t = 0
    if i > 0:
        t+= how_many_traversals(i-1, j)
    if j > 0:
        t+= 1 + how_many_traversals(i, j-1)
    return t

how_many_traversals(1, 1, True)

2

### Hmmm....

Not working.  Let's look at the book's solution.

### Book Solution:

In [2]:
def number_of_ways(n, m):
    def compute_number_of_ways_to_xy(x, y):
        if x == y == 0:
            return 1

        if number_of_ways[x][y] == 0:
            ways_top = 0 if x == 0 else compute_number_of_ways_to_xy(x - 1, y)
            ways_left = 0 if y == 0 else compute_number_of_ways_to_xy(x, y - 1)
            number_of_ways[x][y] = ways_top + ways_left

        return number_of_ways[x][y]

    number_of_ways = [[0] * m for _ in range(n)]
    return compute_number_of_ways_to_xy(n - 1, m - 1)

print(number_of_ways(5,5))

70


### Remarks:

Ah it seems like dynamic programming...  After reviewing with a debugger I found a better way of both debugging 2d arrays and explaining their solution.  

First, I converted the array to pandas dataframe.  This allows me to use PyCharm to debug the array visually.
Next, by stepping through the debugger making a breakpoint each time the number of ways to a cell was calculated, I noticed how the calculation was created.
Finally, I thought of a way of describing the solution in plain English, which will allow me to write a solution for this and similar problems.

For each cell, the number of ways to the cell is the sum of the cell above and the cell to the left.


In [3]:
import pandas as pd
from IPython.display import display
import seaborn as sns

cm = sns.light_palette("yellow", as_cmap=True)

def colored_display(label, cm, df):
    print(label)
    display(df.style.background_gradient(cmap=cm))
    
def solution_1(n, m):
    
    def num_ways_to_cell(x, y):
        if x == y == 0:
            return 1

        if nw.iat[x, y] == 0:
            ways_top = 0 if x == 0 else num_ways_to_cell(x - 1, y)
            ways_left = 0 if y == 0 else num_ways_to_cell(x, y - 1)
            nw.iat[x, y] = ways_top + ways_left
            colored_display("Grid", cm, nw)

        return nw.iat[x, y]


    arr = [[0] * m for _ in range(n)]
    nw = pd.DataFrame(arr)
    colored_display("Empty Grid", cm, nw)
    return num_ways_to_cell(n - 1, m - 1)

print(solution_1(5,5))

Empty Grid


,0,1,2,3,4
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


Grid


,0,1,2,3,4
0,0,1,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


Grid


...

Grid


,0,1,2,3,4
0,0,1,1,1,1
1,1,2,3,4,5
2,1,3,6,10,15
3,1,4,10,20,35
4,1,5,15,35,0


Grid


,0,1,2,3,4
0,0,1,1,1,1
1,1,2,3,4,5
2,1,3,6,10,15
3,1,4,10,20,35
4,1,5,15,35,70


70


### Remarks:

That was an interesting solution but it uses recursion, which is only one way to do things.  Here's my version, which uses nested for loops:



In [4]:
cm = sns.light_palette("cyan", as_cmap=True)

def solution_2(n, m):
    arr = [[0] * m for _ in range(n)]
    grid = pd.DataFrame(arr)
    r, c = grid.shape
    for row in range(r):
        for col in range(c):
            if row == col == 0:
                grid.iat[row, col] = 1
            elif row == 0:
                grid.iat[row, col] = grid.iat[row, col - 1]
            elif col == 0:
                grid.iat[row, col] = grid.iat[row - 1, col]
            else:
                grid.iat[row, col] = grid.iat[row - 1, col] + grid.iat[row, col - 1]
        colored_display("Grid", cm, grid)

    return grid.iat[r - 1, c -1]

print(solution_2(5,5))

Grid


,0,1,2,3,4
0,1,1,1,1,1
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


Grid


,0,1,2,3,4
0,1,1,1,1,1
1,1,2,3,4,5
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


Grid


...

Grid


,0,1,2,3,4
0,1,1,1,1,1
1,1,2,3,4,5
2,1,3,6,10,15
3,1,4,10,20,35
4,0,0,0,0,0


Grid


,0,1,2,3,4
0,1,1,1,1,1
1,1,2,3,4,5
2,1,3,6,10,15
3,1,4,10,20,35
4,1,5,15,35,70


70


### Solution 3
You can make your life easier using numpy and get rid of the unnecessary if/else logic.



In [6]:
import numpy as np

cm = sns.light_palette("pink", as_cmap=True)

def solution_3(n, m):
    arr = np.zeros((n, m))
    arr[:,0] = np.ones(n)
    arr[0,:] = np.ones(m)
    # arr = [[0] * m for _ in range(n)]
    grid = pd.DataFrame(arr)
    r, c = grid.shape
    for row in range(1, r):
        for col in range(1, c):
            grid.iat[row, col] = grid.iat[row -1, col] + grid.iat[row, col - 1]
            colored_display("Grid", cm, grid)
    print(grid)
    return int(grid.iat[n-1, m-1])

print(solution_3(5,5))

Grid


,0,1,2,3,4
0,1,1,1,1,1
1,1,2,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


Grid


,0,1,2,3,4
0,1,1,1,1,1
1,1,2,3,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


Grid


...

Grid


,0,1,2,3,4
0,1,1,1,1,1
1,1,2,3,4,5
2,1,3,6,10,15
3,1,4,10,20,35
4,1,5,15,35,0


Grid


,0,1,2,3,4
0,1,1,1,1,1
1,1,2,3,4,5
2,1,3,6,10,15
3,1,4,10,20,35
4,1,5,15,35,70


     0    1     2     3     4
0  1.0  1.0   1.0   1.0   1.0
1  1.0  2.0   3.0   4.0   5.0
2  1.0  3.0   6.0  10.0  15.0
3  1.0  4.0  10.0  20.0  35.0
4  1.0  5.0  15.0  35.0  70.0
70


### Final Remarks:

The final solution is nice because it does not waste time calculating ones _nor_ does it have to continously check to see if it's in the top row or left column. 

That solution has
Time complexity: $$ O(n \cdot m) $$
Space complexity $$ O(n \cdot m) $$